In [111]:
from news_leadgen.news_handler import NewsHandler
from news_leadgen.dto.newsdto import NewsDTO 
import pandas as pd

In [2]:
%load_ext autoreload 
%autoreload 2

# Чтение вопросов к новостям

In [3]:
with open("questions.txt", "r") as f:
    txt =f.read()
questions = txt.split('\n')

# Чтение новостей из файла

In [40]:
news_data = pd.read_excel("data/Отказы MediaAnalysis_text 10М2023.xlsx", sheet_name=0)[["index", "text"]]
data = news_data.values

all_news = list()
for news_arr in data:
    all_news.append(NewsDTO(news_id=news_arr[0], text=news_arr[1]))

# Обработка новостей

In [34]:
nh = NewsHandler(questions = questions)

In [35]:
print(nh._get_questions_as_one_prompt())

Тебе дан список вопросов и одна новость. Ответь на вопросы, при подготовке ответа не фантазируй. 
        Если не знаешь ответа, так и скажи.
        Вопросы:0) Содержит ли текст новости информацию о строительстве, модернизации или расширении какого-либо промышленного, жилого, коммерческого объекта (или его части) или реализации инфраструктурного, инновационного проекта, который банк может профинансировать, ответь только ДА или НЕТ
1) Если текст новости содержит информацию о строительстве, модернизации или расширении какого-либо промышленного, жилого, коммерческого объекта (или его части) или реализации инфраструктурного, инновационного проекта, который банк может профинансировать, напиши краткое описание новости на 400 символов.
2) Напиши название компании о которой идёт речь в новости
3) Напиши регион о котором идёт речь в новости
4) Напиши ФИО действующих лиц о которых идёт речь в новости
5) Напиши тип инвестиционного проекта: промышленный, жилой, коммерческий, инфраструктурный, инн

## По одной

In [45]:
# возвращается сет: объект с результатом разбора (айди новости, ответ сети) и список заданных сети вопросов

one_news_processing_result, _ = nh.process_news(all_news[1])

/var/folders/jp/ylys3dns1xqg7c6k6bt3cg3h0000gn/T/ipykernel_5509/1911813891.py:3: RuntimeWarning: coroutine 'NewsHandler.process_news' was never awaited
  one_news_processing_result, _ = nh.process_news(all_news[1])


TypeError: cannot unpack non-iterable coroutine object

In [25]:
print(one_news_processing_result.processing_result[0])

1. Главная проблема в Нижнем Тагиле — отток молодежи. Для решения этой проблемы необходимо сделать город более привлекательным для молодежи, предоставить им возможности для получения образования, стабильной работы с хорошим заработком, а также обеспечить доступность яслей и детских садов, школ.
2. Рекордный бюджет города позволит реализовать множество проектов по благоустройству города, развитию туризма, созданию комфортных условий для жителей.
3. Успевать все это удается благодаря опыту и профессионализму руководства города, а также тесному взаимодействию с жителями и поддержке местных предприятий.


Иной подход, предыдущий не работает

In [95]:
text = all_news[260].text

In [96]:
qest = "Тебе нужно коротко ответить на вопрос: 1) Содержит ли текст новости информацию о строительстве, модернизации или расширении какого-либо промышленного, жилого, коммерческого объекта (или его части) или реализации инфраструктурного, инновационного проекта, который банк может профинансировать, ответь только  «ДА» или  «НЕТ»"

In [97]:
response =  NewsHandler().one_news(qest+text)

In [98]:
response

'Да, в тексте новости содержится информация о строительстве нового жилого комплекса бизнес-класса компанией Tekta Group на Дербеневской улице с предварительной оценкой объема инвестиций в размере около 30 млрд. рублей.'

## пачкой

In [46]:
#выделяю часть новостей чтобы не ждать обрабутку всех
news_batch = all_news[:2]

In [47]:
#результат - массив, каждый элемент которого это сет
#Состав сета: объект с резултьатом разбора (айди новости, ответ сети) и список заданных сети вопросов
news_processing_batch = await nh.batch_process_news(news_batch)

In [48]:
npb_list = list()
for npr in news_processing_batch:
    npb_list.append((npr[1], npr[0].news_id, npr[0].processing_result))

In [49]:
responses = pd.DataFrame(data=npb_list, columns=["Заданные вопросы", "ID новости", "Ответы на вопросы"])
responses.head()

,Заданные вопросы,ID новости,Ответы на вопросы
0,0) Содержит ли текст новости информацию о стро...,ef2730c48257bbee6e68354038acd1ee,[1. В прошлом году классные руководители выясн...
1,0) Содержит ли текст новости информацию о стро...,e08e3641050bdabf89c99c518033ff93,[1. Как все успеваете? — Я стараюсь эффективно...


In [20]:
responses.to_excel("responses.xlsx")

## каждый вопрос задается отдельно

In [112]:
import json 

In [113]:
with open('data/question.json') as json_data:
    questions_json = json.load(json_data)
    json_data.close()


In [117]:
from asyncio import gather
from news_leadgen.news_handler import NewsHandler

In [115]:
nh = NewsHandler()

In [118]:
ms =  await nh.q(all_news[259:261], questions_json)

In [119]:
ms

[{'news_id': '484a3456eb352fb42fc5685fa6684a50',
  'Messeges_array': [Messages(role='system', content='Отвечай коротко. Не более трех букв на ответ.'),
   Messages(role='assistant', content='ДА'),
   Messages(role='assistant', content='НЕТ'),
   Messages(role='user', content='Тебе нужно коротко ответить на вопрос: 1) Содержит ли текст новости информацию о строительстве, модернизации или расширении какого-либо промышленного, жилого, коммерческого объекта (или его части) или реализации инфраструктурного, инновационного проекта, который банк может профинансировать, ответь только  «ДА» или  «НЕТ». Текст Новости:  15 февраля электросети проведут плановые работы в Новой Слободке Плавского района. 15.02.2023 с 9.00 до 17.00 запланировано отключение электроснабжения в деревне Новая Слободка. Бригада Плавского РЭС Тулэнерго проведет расчистку участка высоковольтной линии электропередач от дикорастущей поросли ')],
  'step': '1',
  'number': 1,
  'answer': 'Да'},
 {'news_id': '484a3456eb352fb42f

In [121]:
ms_pd = pd.DataFrame.from_dict(ms)

In [124]:
ms_pd.pivot(index= 'news_id', columns = 'number', values = 'answer')

number,1,2,3,4,5,6,8,9,10,11,12,13
news_id,,,,,,,,,,,,
484a3456eb352fb42fc5685fa6684a50,Да,Электросети планируют провести плановые работы...,"Название компании, упомянутой в новости — ""Тул...","В новости говорится о плановых работах, которы...",Коммерческий проект.,В новости упоминаются следующие действующие ли...,"В данном тексте нет информации о числе, которо...",Событие новости происходит в настоящем времени.,Год события: 2023.,"Временной период события, описанного в новости...",Да,"Из текста новости не указано, за чьи средства ..."
65286f691e018a9e76d68908d43b9b3b,Да,Компания Tekta Group планирует построить новый...,Название компании в данной новости — Tekta Group.,В данной новости речь идет о строительстве нов...,Коммерческий,В новости упоминается компания Tekta Group и е...,Это будет 30 миллиардов рублей.,Событие новости происходит в настоящем времени.,Год события: 2023,Временной период события — 27.03.2023.,Да,Компания Tekta Group.


In [120]:
pd.DataFrame.from_dict(ms).to_excel('output.xlsx')